In [64]:
import os
import pandas as pd
import requests
from dotenv import load_dotenv

from search_para import search_para
from search_parser import extract_doc_urls
from doc_parser import extract_profile_info

load_dotenv()

True

In [65]:
# Bypass token verification
cookies = requests.cookies.RequestsCookieJar()
cookies['__RequestVerificationToken_L1B1YmxpYw2'] = os.environ.get(
    "__RequestVerificationToken_L1B1YmxpYw2")


headers = {
    'User-Agent': 'Mozilla/5.0',
    'Content-Type': 'application/x-www-form-urlencoded',
    'sec-ch-ua': '"Chromium";v="122", "Not(A:Brand";v="24", "Google Chrome";v="122"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'Upgrade-Insecure-Requests': '1'
}

In [66]:
def search_doc(doc_name):
    print("Doc to search:", doc_name)

    # Import search parameters
    request_para = search_para
    request_para['Name'] = doc_name
    request_para['__RequestVerificationToken'] = os.environ.get(
    "__RequestVerificationToken")

    # Sending POST request with the specified headers and data
    search_url = 'https://apps.pcdirectory.gov.hk/Public/TC/AdvancedSearch'
    search_response = requests.post(search_url, headers=headers,
                            cookies=cookies, data=request_para)

    # Save if successfully get search reseults
    if search_response.status_code == 200:
        # Save search result conrent as HTML file
        with open('temp/search_response.html', 'wb') as f:
            f.write(search_response.content)
        print("Search results saved as HTML file.")
    else:
        print("Failed to retrieve search results.")

    # Extract doc profile urls
    print("\nParsing search results...")
    doc_url_lst = extract_doc_urls()

    return doc_url_lst

In [72]:
# ======================
# Extract Doc Profile
# ======================

def extract_doc_detail(doc_url_lst):
    doc_profile_lst = []


    print("\nExtracting doc info from profile...")
    doc_counter = 0
    for doc_url in doc_url_lst:
        doc_counter += 1
        doc_profile_response = requests.get(doc_url, headers=headers, cookies=cookies)

        print("\n" + "-" * 30 + "\n")
        print(f"Processing doc profile {doc_counter}")

        # Save doc profile as HTML
        if doc_profile_response.status_code == 200:
            with open(f'temp/doc_profile_{doc_counter}.html', 'wb') as f:
                f.write(doc_profile_response.content)
            print(f"Doc profile ({doc_counter}) saved as HTML file.")
        else:
            print(f"Failed to access doc profile {doc_url}")
            continue

        doc_profile_info = extract_profile_info(doc_counter)
        doc_profile_lst.append(doc_profile_info)
        
        # Info summary check (for debug)
        print("Required fields in doc profile extracted:\n")
        for key, value in doc_profile_info.items():
            print(key, ':', value)

    return doc_profile_lst

# Driver Code

In [73]:
doc_names = ["竺培鎮", "陳計明"]

In [74]:
result_profiles = []

for doc_name in doc_names:
    print("\n" + "=" * 30 + "\n")
    doc_url_lst = search_doc(doc_name)
    doc_profile_lst = extract_doc_detail(doc_url_lst)

    result_profiles += doc_profile_lst



Doc to search: 竺培鎮
Search results saved as HTML file.

Parsing search results...
(1) Found doc name: 竺培鎮, link: https://apps.pcdirectory.gov.hk/Public/TC/SearchResult/ToViewDetails?DPID=00104026&ProfID=RMP&PracticeID=1&SelectedResultRowID=1

Extracting doc info from profile...

------------------------------

Processing doc profile 1
Doc profile (1) saved as HTML file.
Required fields in doc profile extracted:

姓名 : 竺培鎮
性別 : 男性
電郵 : 没有提供
基層醫療服務提供者類別 : 西醫
香港醫務委員會註冊號碼 : M01161
執業處所 : 西醫竺培鎮
地址 : 香港灣仔柯布連道3B號3樓
執業類別 : 私營
電話 : 25740088
政府基層醫療促進計劃 : 長者醫療券計劃 (HCVS), 疫苗資助計劃 (VSS), 院舍防疫注射計劃 (RVP)


Doc to search: 陳計明
Search results saved as HTML file.

Parsing search results...
(1) Found doc name: 陳計明, link: https://apps.pcdirectory.gov.hk/Public/TC/SearchResult/ToViewDetails?DPID=00105942&ProfID=RMP&PracticeID=1&SelectedResultRowID=1
(2) Found doc name: 陳計明, link: https://apps.pcdirectory.gov.hk/Public/TC/SearchResult/ToViewDetails?DPID=00105942&ProfID=RMP&PracticeID=2&SelectedResultRowID=2



In [75]:
result_df = pd.DataFrame(result_profiles)
result_df

,姓名,性別,電郵,基層醫療服務提供者類別,香港醫務委員會註冊號碼,執業處所,地址,執業類別,電話,政府基層醫療促進計劃,科別,應診時間
0,竺培鎮,男性,没有提供,西醫,M01161,西醫竺培鎮,香港灣仔柯布連道3B號3樓,私營,25740088,"長者醫療券計劃 (HCVS), 疫苗資助計劃 (VSS), 院舍防疫注射計劃 (RVP)",NaN,NaN
1,陳計明,男性,没有提供,西醫,M13160,善心醫務中心,香港北角天后電氣道134號泰華大廈地舖,私營,25700012,"大腸癌篩查計劃 (CRCSP), 長者醫療券計劃 (HCVS), 疫苗資助計劃 (VSS),...",普通科,星期一(09:00 至 13:30) (15:30 至 20:00)\n星期二(09:00 ...
2,陳計明,男性,没有提供,西醫,M13160,西醫陳計明,香港銅鑼灣興發街36號地舖,私營,25700012,"大腸癌篩查計劃 (CRCSP), 長者醫療券計劃 (HCVS), 疫苗資助計劃 (VSS),...",普通科,星期一(12:00 至 13:00) (15:30 至 18:00)\n星期二(12:00 ...


In [76]:
result_df.to_excel('input_output/doc_profile_result.xlsx', index=False)